In [1]:
import torch
from train_pointnet import VesselData, VAEEncoder, VAEDecoder, Pointnet, DATA_PATH, MODEL_PATH, TEST_VESSELS

In [2]:
# Hyperparameters
SAMPLE_SIZE = 2048

ENABLE_TRANSFORM = True
NUM_EPOCHS = 100
BATCH_SIZE = 4
LEARNING_RATE = 0.01
SPLIT_RATIO = 0.8
LOG_EVERY = 5
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_WORKERS = 2
SAVE_MODEL = True

def create_model_name_infix(dict):
    infix = "_"
    for key, value in dict.items():
        if '.' in str(value):
            value = str(value).split('.')[1]
        infix += f"_{key}_{value}"
    return infix

model_dict = {
    'lr': LEARNING_RATE,
    'ep': NUM_EPOCHS,
    'transf': ENABLE_TRANSFORM,
}

SAVE_MODEL_NAME = 'single_coord_linear_encoder' + create_model_name_infix(model_dict) + '.pth'

In [3]:
vessel_dataset = VesselData(DATA_PATH, num_samples=SAMPLE_SIZE, enable_transform=ENABLE_TRANSFORM)

In [4]:
vessel_dataset[0][0].shape

torch.Size([2048, 3])

In [5]:
import os
import numpy as np

vessel_number = 0

vessel_array = np.load(os.path.join(DATA_PATH, TEST_VESSELS[vessel_number]), allow_pickle=True)
input_array = vessel_array['fluid_points']
target_array = vessel_array['sys_vel']
input_array.shape, target_array.shape

((104492, 3), (104492, 3))

In [15]:
input_array, target_array = vessel_dataset.transform(
    torch.Tensor(input_array), 
    torch.Tensor(target_array)
)
input_array.shape, target_array.shape

(torch.Size([104492, 3]), torch.Size([104492, 3]))

In [16]:
# Assuming input_array is the tensor you want to reshape
B = input_array.shape[0] // SAMPLE_SIZE
print(B)

batched_input_tensor = torch.Tensor(input_array)[:B*SAMPLE_SIZE].view(B, SAMPLE_SIZE, 3)
batched_input_tensor.shape

51


torch.Size([51, 2048, 3])

In [17]:
batched_target_tensor = torch.Tensor(target_array)[:B*SAMPLE_SIZE].view(B, SAMPLE_SIZE, 3)
batched_target_tensor.shape

torch.Size([51, 2048, 3])

In [18]:
# Load the model
encoder = VAEEncoder(in_channels=3, z_size=64)
decoder = VAEDecoder(z_size=64, out_dim=SAMPLE_SIZE)
model = Pointnet(encoder=encoder, decoder=decoder)
model.load_state_dict(torch.load(os.path.join(MODEL_PATH, SAVE_MODEL_NAME), map_location=DEVICE))
model.eval()

/tmp/ipykernel_3214210/964596758.py:5: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



Pointnet(
  (encoder): VAEEncoder(
    (classifier): Sequential(
      (0): Linear(in_features=64, out_features=2, bias=True)
    )
    (embedding_layer): Embedding(2, 64)
    (conv): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): ReLU(inplace=True)
      (2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (3): ReLU(inplace=True)
      (4): Conv1d(128, 256, kernel_size=(1,), stride=(1,))
      (5): ReLU(inplace=True)
      (6): Conv1d(256, 256, kernel_size=(1,), stride=(1,))
      (7): ReLU(inplace=True)
      (8): Conv1d(256, 512, kernel_size=(1,), stride=(1,))
    )
    (fc): Sequential(
      (0): Linear(in_features=512, out_features=256, bias=True)
      (1): ReLU(inplace=True)
    )
    (mu_layer): Linear(in_features=256, out_features=64, bias=True)
    (std_layer): Linear(in_features=256, out_features=64, bias=True)
  )
  (decoder): VAEDecoder(
    (model): Sequential(
      (0): Linear(in_features=64, out_features=64, bias=True)
      (1): R

In [19]:
pred = model(batched_input_tensor)
pred.shape

torch.Size([51, 2048, 3])

In [20]:
input_sample = batched_input_tensor.detach().numpy()
target_sample = batched_target_tensor.detach().numpy()
pred_sample = pred.detach().numpy()

input_sample = input_sample.reshape(-1, 3)
target_sample = target_sample.reshape(-1, 3)
pred_sample = pred_sample.reshape(-1, 3)

vis_indices = np.random.choice(input_sample.shape[0], 1000, replace=False)

input_sample = input_sample[vis_indices]
target_sample = target_sample[vis_indices]
pred_sample = pred_sample[vis_indices]


input_sample.shape, target_sample.shape, pred_sample.shape

((1000, 3), (1000, 3), (1000, 3))

In [21]:
#pred_sample = np.around(pred_sample.clip(-5, 5), decimals=3)
#pred_sample

In [22]:
import plotly.graph_objects as go

xyz = input_sample[:,:3]

fig = go.Figure(data=[go.Cone(
    x=xyz[:,0], 
    y=xyz[:,1], 
    z=xyz[:,2],
    u=pred_sample[:,0], 
    v=pred_sample[:,1], 
    w=pred_sample[:,2],
    colorscale='Viridis',
    sizemode='absolute',
    sizeref=1,  # Increase the size reference to make the cones bigger
    showscale=True
    )])
fig.update_layout(scene=dict(
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='Z Axis'),
            title="Prediction",
            height=800,
            width=800)
fig.show()

fig2 = go.Figure(data=[go.Cone(
    x=xyz[:,0], 
    y=xyz[:,1], 
    z=xyz[:,2],
    u=target_sample[:,0], 
    v=target_sample[:,1], 
    w=target_sample[:,2],
    colorscale='Viridis',
    sizemode='absolute',
    sizeref=2,
    showscale=True
    )])
fig2.update_layout(scene=dict(
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='Z Axis'),
            title="Target",
            height=800,
            width=800)
fig2.show()

In [23]:
import torch.nn.functional as F

# Calculate MSE loss
mse_loss = F.mse_loss(torch.Tensor(pred_sample), torch.Tensor(target_sample)).item()
print(f'MSE Loss: {mse_loss}')

# Visualize the MSE loss in a 3D scatter plot
mse_values = ((pred_sample - target_sample) ** 2).mean(axis=1)

fig3 = go.Figure(data=[go.Scatter3d(
    x=xyz[:, 0],
    y=xyz[:, 1],
    z=xyz[:, 2],
    mode='markers',
    marker=dict(
        size=5,
        color=mse_values,  # set color to the MSE values
        colorscale='Viridis',
        colorbar=dict(title='MSE'),
        opacity=0.8
    )
)])
fig3.update_layout(scene=dict(
            xaxis_title='X Axis',
            yaxis_title='Y Axis',
            zaxis_title='Z Axis'),
            title="3D Scatter Plot of MSE Loss",
            height=800,
            width=800)
fig3.show()

MSE Loss: 0.12113601714372635


In [24]:
import torch.nn.functional as F

def cosine_similarity(point_cloud1, point_cloud2):
    cos_sim = F.cosine_similarity(
        torch.Tensor(point_cloud1), 
        torch.Tensor(point_cloud2), 
        dim=1)
    
    return cos_sim.mean().item()

def mse_loss(point_cloud1, point_cloud2):
    mse = F.mse_loss(
        torch.Tensor(point_cloud1), 
        torch.Tensor(point_cloud2))
    
    return mse.item()

In [34]:
cos_sim_list = []
mse_list = []
for v in TEST_VESSELS:
    vessel_array = np.load(os.path.join(DATA_PATH, v), allow_pickle=True)
    input_array = torch.Tensor(vessel_array['fluid_points'])
    target_array = torch.Tensor(vessel_array['sys_vel'])
    
    centroid = input_array.mean(dim=0)
    translated_points = input_array - centroid
    max_abs_val = torch.max(torch.abs(translated_points))
    scaled_points = translated_points / (2 * max_abs_val)
    input_array = scaled_points + 0.5
    
    print(input_array)
    
    B = input_array.shape[0] // SAMPLE_SIZE
    batched_input_tensor = torch.Tensor(input_array)[:B*SAMPLE_SIZE].view(B, SAMPLE_SIZE, 3)
    batched_target_tensor = torch.Tensor(target_array)[:B*SAMPLE_SIZE].view(B, SAMPLE_SIZE, 3)
    
    print(batched_input_tensor.shape, batched_target_tensor.shape)
    
    
    batched_pred_tensor = model(batched_input_tensor)
    
    pred_tensor = batched_pred_tensor.reshape(B, 3)
    
    cos_sim = cosine_similarity(pred_tensor, target_array)
    mse = mse_loss(pred_tensor, target_array)
    print(f'Vessel: {v}, Cosine Similarity: {cos_sim}, MSE Loss: {mse}')
    
    cos_sim_list.append(cos_sim)
    mse_list.append(mse)
    
cos_sim_list = np.array(cos_sim_list)
mse_list = np.array(mse_list)

print(f'Mean Cosine Similarity: {cos_sim_list.mean()}')
print(f'Mean MSE Loss: {mse_list.mean()}')

tensor([[0.4485, 0.5430, 0.5959],
        [0.4486, 0.5428, 0.5957],
        [0.4482, 0.5427, 0.5958],
        ...,
        [0.3132, 0.4925, 0.6194],
        [0.3103, 0.4832, 0.6245],
        [0.3106, 0.4883, 0.6222]])
torch.Size([51, 2048, 3]) torch.Size([51, 2048, 3])


RuntimeError: shape '[51, 3]' is invalid for input of size 313344

In [38]:
# Evaluate the model
model.eval()

import pandas as pd
from utils.metrics import *
import meshio

# load data
pinn_path = "/home/ne34gux/workspace/experiments/data/pinn_blood_flow_geometries"
test_vessel_files = [os.path.join(DATA_PATH, f) for f in TEST_VESSELS]
test_vessel_files

test_geometry_files = [os.path.join(pinn_path, f) for f in os.listdir(pinn_path)]
test_geometry_files

results = []

for g in TEST_VESSELS:
    g_object = np.load(os.path.join(DATA_PATH, g))
    xyz = torch.Tensor(g_object['fluid_points'])
    vel = torch.Tensor(g_object['sys_vel'])
    #geom = torch.Tensor(g_object['mesh_points'])
    
    centroid = xyz.mean(dim=0)
    translated_points = xyz - centroid
    max_abs_val = torch.max(torch.abs(translated_points))
    scaled_points = translated_points / (2 * max_abs_val)
    xyz = scaled_points + 0.5

    B = xyz.shape[0] // SAMPLE_SIZE
    batched_input_tensor = torch.Tensor(xyz)[:B*SAMPLE_SIZE].view(B, SAMPLE_SIZE, 3)
    batched_target_tensor = torch.Tensor(vel)[:B*SAMPLE_SIZE].view(B, SAMPLE_SIZE, 3)
    #rand_idx = np.random.choice(geometry_array.shape[0], size=ENCODER_SAMPLE_SIZE, replace=False)
    #batched_geometry_tensor = geom[rand_idx].repeat(B, 1, 1)
    
    batched_pred_tensor = model(batched_input_tensor)

    mse = mse_error(batched_pred_tensor, batched_target_tensor)
    mae = mae_error(batched_pred_tensor, batched_target_tensor)
    cs = cosine_similarity(batched_pred_tensor, batched_target_tensor)

    print(g)
    print(f"   MSE: {mse}")
    print(f"   MAE: {mae}")
    print(f"   Cosine Similarity: {cs}")
    
    results.append([g.split(".")[0], mse, mae, cs])
    
for g in os.listdir(pinn_path):
    if "velocities" not in g:
        continue
    g_object = pd.read_csv(os.path.join(pinn_path, g)).to_numpy()
    xyz = torch.Tensor(g_object[:,4:7])
    vel = torch.Tensor(g_object[:,0:3])
    
    centroid = xyz.mean(dim=0)
    translated_points = xyz - centroid
    max_abs_val = torch.max(torch.abs(translated_points))
    scaled_points = translated_points / (2 * max_abs_val)
    xyz = scaled_points + 0.5
    
    wall_file = g.replace("_velocities.csv", "_wall.stl")
    wall = meshio.read(os.path.join(pinn_path, wall_file)).points
    #rand_idx = np.random.choice(wall.shape[0], size=ENCODER_SAMPLE_SIZE, replace=False)
    #B = xyz.shape[0] // SAMPLE_SIZE
    
    B = xyz.shape[0] // SAMPLE_SIZE
    batched_input_tensor = torch.Tensor(xyz)[:B*SAMPLE_SIZE].view(B, SAMPLE_SIZE, 3)
    batched_target_tensor = torch.Tensor(vel)[:B*SAMPLE_SIZE].view(B, SAMPLE_SIZE, 3)
    #batched_geometry_tensor = torch.Tensor(wall[rand_idx]).repeat(B, 1, 1)
    
    batched_pred_tensor = model(batched_input_tensor)

    mse = mse_error(batched_pred_tensor, batched_target_tensor)
    mae = mae_error(batched_pred_tensor, batched_target_tensor)
    cs = cosine_similarity(batched_pred_tensor, batched_target_tensor)

    print(g)
    print(f"   MSE: {mse}")
    print(f"   MAE: {mae}")
    print(f"   Cosine Similarity: {cs}")
    
    results.append([g.split(".")[0], mse, mae, cs])    

# save results
result_path = "/home/ne34gux/workspace/experiments/results"
results_df = pd.DataFrame(data=results, columns=["Vessel", "MSE", "MAE", "Cosine_Similarity"])
results_df.to_csv(
    os.path.join(result_path, SAVE_MODEL_NAME.replace(".pth", ".csv")), 
    index=False
)
print(f"Results saved to {os.path.join(result_path, SAVE_MODEL_NAME.replace('.pth', '.csv'))}")

case_k_003_left.npz
   MSE: 0.1283840835094452
   MAE: 0.20043011009693146
   Cosine Similarity: 0.2661171555519104
case_k_008_right.npz
   MSE: 0.028703369200229645
   MAE: 0.11707872152328491
   Cosine Similarity: 0.3720775544643402
case_m_006_left.npz
   MSE: 0.022693827748298645
   MAE: 0.1106833964586258
   Cosine Similarity: 0.5872010588645935
case_w_027_right.npz
   MSE: 0.038436949253082275
   MAE: 0.13941636681556702
   Cosine Similarity: 0.2481110841035843
case_w_038_left.npz
   MSE: 0.05343350023031235
   MAE: 0.16193051636219025
   Cosine Similarity: 0.40976181626319885


/home/ne34gux/workspace/experiments/.venv/lib64/python3.9/site-packages/meshio/stl/_stl.py:40: RuntimeWarning:

overflow encountered in scalar multiply



aneurysm1_velocities.csv
   MSE: 0.029059486463665962
   MAE: 0.12825201451778412
   Cosine Similarity: 0.003183700144290924
aneurysm2_velocities.csv
   MSE: 0.0188140831887722
   MAE: 0.09741061180830002
   Cosine Similarity: -0.07388292998075485


/home/ne34gux/workspace/experiments/.venv/lib64/python3.9/site-packages/meshio/stl/_stl.py:40: RuntimeWarning:

overflow encountered in scalar multiply



bifurication_velocities.csv
   MSE: 0.018348993733525276
   MAE: 0.10726357996463776
   Cosine Similarity: 0.026054857298731804


/home/ne34gux/workspace/experiments/.venv/lib64/python3.9/site-packages/meshio/stl/_stl.py:40: RuntimeWarning:

overflow encountered in scalar multiply



cylinder_velocities.csv
   MSE: 0.02313549630343914
   MAE: 0.12276367098093033
   Cosine Similarity: 0.028998013585805893
Results saved to /home/ne34gux/workspace/experiments/results/single_coord_linear_encoder__lr_01_ep_100_transf_True.csv
